## SDP with group structures: A Demo

In [1]:
import numpy as np
import cvxpy as cvx

(CVXPY) Jul 27 04:19:46 PM: Encountered unexpected exception importing solver SCS:
ImportError('dlopen(/Users/yibin/.local/lib/python3.7/site-packages/scs-3.2.0-py3.7-macosx-10.9-x86_64.egg/_scs_direct.cpython-37m-darwin.so, 2): Symbol not found: _aligned_alloc\n  Referenced from: /Users/yibin/.local/lib/python3.7/site-packages/scs-3.2.0-py3.7-macosx-10.9-x86_64.egg/scs/.dylibs/libgomp.1.dylib (which was built for Mac OS X 10.15)\n  Expected in: /usr/lib/libSystem.B.dylib\n in /Users/yibin/.local/lib/python3.7/site-packages/scs-3.2.0-py3.7-macosx-10.9-x86_64.egg/scs/.dylibs/libgomp.1.dylib')


In [2]:
n = 500
p = 100
# generate a symmetric PSD matrix for covariance matrix
np.random.seed(0)
data = np.random.randn(n,p)
Sigma = data.T @ data / n
sqrtDiagSigma = np.sqrt(np.diag(Sigma))
scalingFactors = np.outer(sqrtDiagSigma,sqrtDiagSigma)
corrMatrix = np.divide(Sigma, scalingFactors)

print(f'The smallest eigenvalue of corrMatrix is {np.linalg.eigvals(corrMatrix)[-1]}')

The smallest eigenvalue of corrMatrix is 0.8698219203277063


In [3]:
cat_name_to_idx = {'BsmtFinType2': [36, 37, 38, 39, 40], 'HeatingQC': [41, 42], 
'Neighborhood': [43, 44, 45, 46], 'SaleType': [47, 48, 49, 50], 
'Condition2': [51, 52], 'GarageFinish': [53, 54, 55, 56, 57], 
'LandContour': [58, 59, 60], 
'BsmtFinType1': [61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85]}

In [4]:
# build a more conveninent data structure to encode group size information
# group_sizes contains the numbers of dummy variables in each group. If a variable is numerical, then its group size = 1
group_sizes = []
for i in range(p):
    # a flag that indicates if a variable is numerical or categorical
    num = True
    for group_indices in cat_name_to_idx.values():
        if i == group_indices[0]:
            num = False
            group_sizes.append(len(group_indices))
            break
        elif i in group_indices:
            num = False
            break
    # numerical variables
    if num:
        group_sizes.append(1)
print(f'Number of variables in SDP: {len(group_sizes)}')

Number of variables in SDP: 58


#### Get the block-diagonal correlation matrix

In [5]:
# define a mask matrix that helps us only keep the block-diagonal entries
mask_mat = np.zeros((p,p))
i = 0
for g_size in group_sizes:
    mask_mat[i:i+g_size,i:i+g_size] = np.ones((g_size,g_size))
    i += g_size
block_corr = corrMatrix * mask_mat
block_corr[35:40,35:40]

array([[ 1.        ,  0.        ,  0.        , -0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.0523586 , -0.0273276 , -0.01457778],
       [ 0.        ,  0.0523586 ,  1.        ,  0.05688207,  0.00165129],
       [-0.        , -0.0273276 ,  0.05688207,  1.        ,  0.01160882],
       [ 0.        , -0.01457778,  0.00165129,  0.01160882,  1.        ]])

### Set up the SDP problem

\begin{align*}
    \min_{\gamma_j}&\, \sum_{j=1}^m\,(1-\gamma_j)\,\|\Sigma_{G_j,G_j}\|_F \\
    \text{s.t.}&\, 0\leq \gamma_j \leq 1\hspace{0.2cm}\forall\,j, \\
    &\,S \preceq 2\Sigma
\end{align*}

In [6]:
m = len(group_sizes) # number of groups
gamma = cvx.Variable(m)

Challenge: We want to repeat each $\gamma_j$ for $|G_j|$ times to build diag $\{\gamma_1\cdot I_{G_1,G_1},\dots, \gamma_m\cdot I_{G_m,G_m}\}$ (this is `gamma_mat` in my code). I seemed to mess up NumPy objects and cvxpy objects, which are symbolic expressions. How can we do this step? I can't assign values to a cvxpy object

An attempt to aviod *assigning numbers to part of a matrix* was that I wanted to use Kronecker product to represent diag $\{\gamma_1\cdot I_{G_1,G_1},\dots, \gamma_m\cdot I_{G_m,G_m}\}$, but the group sizes $|G_j|$ are different over all $j$'s. Thus, we cannot use Kronecker product.

### Challenge solved: check the code below. Just don't do value assignment on cvxpy object and use `vstack` function

In [7]:
var_list = []
for i in range(len(group_sizes)):
    var_list += [gamma[i]] * group_sizes[i]
gamma_full = cvx.vstack(var_list)
gamma_full

Expression(AFFINE, UNKNOWN, (100, 1))

In [8]:
# define the S matrix
S = cvx.diag(gamma_full) @ block_corr
# define the objective
objective = 0
i = 0
for j in range(len(group_sizes)):
    g_size = group_sizes[j]
    objective += (1-gamma[j]) * cvx.norm(block_corr[i:i+g_size, i:i+g_size],'fro')
    i += g_size

In [9]:
obj = cvx.Minimize(objective)
constraints = [0 <= gamma, gamma <= 1, S << 2*corrMatrix]
prob = cvx.Problem(obj,constraints)
prob.solve()

print(f'Status: {prob.status}')
print(f'Objective value: {prob.value}')
print(f'Solution: gamma = {gamma.value}')

Status: optimal
Objective value: 15.552269198728547
Solution: gamma = [0.61035965 0.76062037 0.85911258 0.82308662 0.89156198 0.95871633
 0.99478335 0.70325079 0.60294812 0.99999991 0.91724343 0.55545384
 0.67745898 0.75456202 0.94604387 0.99999954 0.99999999 0.7391584
 0.76918968 0.7312014  0.92918561 0.99999998 0.86825065 0.41743115
 0.97379255 0.88707274 0.85483287 0.99997936 0.72540849 0.64749562
 0.99999995 0.66482039 0.99631604 0.89895887 0.98073053 0.77546397
 0.57073804 0.99851185 0.66891933 0.5820442  0.89695115 0.63120891
 0.66109657 0.41653886 0.92021027 0.99999989 0.3137507  0.68907948
 0.97602408 0.90210932 0.99999995 0.90205374 0.96809005 0.81110004
 0.41495043 0.87938932 0.99999997 0.77324287]


### Another way to do this: 

#### we define the variables to be a $p$-dimensional vector rather than $m$-dimensional and impose the constraints that variables that corresponds to indices in the same group are equal

In [10]:
gamma = cvx.Variable(p) # although in essence we only need m variables
# define the S matrix by element-wise multiply block_corr with the vector gamma
S = cvx.multiply(block_corr,cvx.diag(gamma))

In [11]:
# enforce the "group" constraints that the gammas whose indices corresponds to the same group are equal
group_constraints = []
i = 0
for group in cat_name_to_idx.values():
    group_constraints += [gamma[group[0]] == gamma[group[k]] for k in range(1,len(group)) ]
print(f'{len(group_constraints)} constraints defined, expect {4+1+3+3+1+4+2+24} constraints') # this is correct

# define the constraints in the SDP
constraints = [0<= gamma, gamma <=1, S << 2*corrMatrix]
constraints += group_constraints

42 constraints defined, expect 42 constraints


In [12]:
# define the objective
sum_fro_norm = 0
i = 0
for g_size in group_sizes:
    sum_fro_norm += cvx.norm((block_corr - S)[i:i+g_size, i:i+g_size], 'fro')
    i += g_size
objective = cvx.Minimize(sum_fro_norm)
# solve the problem
prob = cvx.Problem(objective,constraints)
prob.solve()

print(f'Status: {prob.status}')
print(f'Objective value: {prob.value}')
print(f'Solution: gamma = {gamma.value}')

Status: optimal
Objective value: 15.94751842623297
Solution: gamma = [0.57957717 0.7649518  0.85088558 0.83702769 0.86881141 0.94343938
 0.99340031 0.71095925 0.62221295 0.99999994 0.91847021 0.58149478
 0.72000486 0.7887077  0.95660652 0.99999994 0.99999999 0.73910971
 0.79316246 0.74136225 0.97998585 0.99999998 0.86589152 0.41965952
 0.97836281 0.89553597 0.8639323  0.99609985 0.77055224 0.70050357
 0.99999994 0.66014566 0.9999998  0.88243876 0.98667278 0.74520355
 0.59287008 0.59287008 0.59287008 0.59287008 0.59287008 0.98791686
 0.98791686 0.63778622 0.63778622 0.63778622 0.63778622 0.57795536
 0.57795536 0.57795536 0.57795536 0.93713355 0.93713355 0.64779992
 0.64779992 0.64779992 0.64779992 0.64779992 0.6845372  0.6845372
 0.6845372  0.32930247 0.32930247 0.32930247 0.32930247 0.32930247
 0.32930247 0.32930247 0.32930247 0.32930247 0.32930247 0.32930247
 0.32930247 0.32930247 0.32930247 0.32930247 0.32930247 0.32930247
 0.32930247 0.32930247 0.32930247 0.32930247 0.32930247 0.329